In [1]:
import json

In [11]:
f = open('mpqa_csds.json', 'r', encoding='utf-8')

In [12]:
data = json.load(f)

In [119]:
import re

def replace_punc(clean_text, offset, offset_list):
    s = clean_text
    s, off1 = re.subn(' \.', '.', s)
    s, off2 = re.subn(' ,', ',', s)
    s, off3 = re.subn(' ;', ';', s)
    s, off4 = re.subn(' :', ':', s)
    s, off5 = re.subn(' \'s', '\'s', s)
    offset -= (off1+off2+off3+off4+off5)
    if (off1+off2+off3+off4+off5 > 0):
        offset_list[-1] -=1
    return s, offset, offset_list

In [120]:
def assemble_tokens(w_text):
    clean_text = ''
    offset = 0
    offset_list = [0]
    index = 0
    in_quote = False
    while index < len(w_text):
        if (index == 0):
            clean_text += w_text[index]
            offset += len(w_text[index])
            index+=1
            continue
        if w_text[index] == '"' or w_text[index] == '\'':
            if not in_quote:
                clean_text += ' ' + w_text[index] + w_text[index+1]
                offset += 1
                offset_list.append(offset)
                offset+= len(w_text[index])
                offset_list.append(offset)
                #for the word itself
                index+=1
                offset += len(w_text[index])
                in_quote = True
            else:
                clean_text += w_text[index]
                offset_list.append(offset)
                offset += 1
                in_quote = False
        else:
            clean_text += ' '
            offset += 1
            offset_list.append(offset)
            clean_text += w_text[index]    
            offset += len(w_text[index])
        clean_text, offset, offset_list = replace_punc(clean_text, offset, offset_list)
        index += 1
    return clean_text, offset_list

In [122]:
limit = 100
count = 0
punc_list = ['.', ',', ';', ':', '\"', '\'']
for obj in data:
    if count > limit:
        break
    s1 = set(['w_head_span', 'w_text', 'clean_head', 'head_start', 'head_end'])
    # for ind in range(len(obj['w_text'])):
    #     if obj['w_text'][ind] == '\'s':
    #         obj['w_text'][ind] = "POSSESSIVE"
    if s1.issubset(obj.keys()):
        clean, offset_list = assemble_tokens(obj['w_text'])
        w_head, w_end = obj['w_head_span']
        # print(clean)
        print('ORIGINAL:', obj['clean_head'])
        s_head = offset_list[w_head]
        e_head = offset_list[w_end]
        if w_head == 0 and w_end == 0:
            print('     NEW:', '')
        elif offset_list[-1] == e_head or clean[offset_list[w_end+1]-2] in punc_list or clean[offset_list[w_end+1]-1] in punc_list:
            print('     NEW:', clean[s_head:e_head])
        else:
            print('     NEW:', clean[s_head:e_head-1])
    count+=1

ORIGINAL: branded as "evil "
     NEW: branded as "evil"
ORIGINAL: "nuclear club "
     NEW: "nuclear club"
ORIGINAL: may have erred
     NEW: may have erred
ORIGINAL: more clearly
     NEW: more clearly
ORIGINAL: real
     NEW: real
ORIGINAL: as far as human rights are concerned
     NEW: as far as human rights are concerned
ORIGINAL: easily qualify as repressive and violating human rights
     NEW: easily qualify as repressive and violating human rights
ORIGINAL: oppressive
     NEW: oppressive
ORIGINAL: not in itself
     NEW: not in itself
ORIGINAL: aggressively
     NEW: aggressively
ORIGINAL: evildoer
     NEW: evildoer
ORIGINAL: therefore
     NEW: therefore
ORIGINAL: scores of
     NEW: scores of
ORIGINAL: seemingly
     NEW: seemingly
ORIGINAL: Perhaps
     NEW: Perhaps
ORIGINAL: In other words
     NEW: In other words
ORIGINAL: aggressively
     NEW: aggressively
ORIGINAL: grave human rights violators
     NEW: grave human rights violators
ORIGINAL: irrespective
     NEW: irr

What to do with possesives? They are considered as own token For example: PRESIDENT GEORGE Bush 's National Security Adviser Condoleezza Rice has recently "defined" the context and scope of the term "axis of evil" that Bush used in his State of the Union address a few weeks ago to describe Iraq, Iran and North Korea.

Replace single quote with sequence of distinct character ("POSSESSIVE") and then replace after with regular expression substitution

New output: clean sentence and a list of character offsets (start index) for each token, then use those indices for each to calculate for head/anchor offsets

In [72]:
w_text = [
                "As",
                "for",
                "her",
                "wording",
                "regarding",
                "acquisition",
                "of",
                "weapons",
                ",",
                "what",
                "is",
                "particularly",
                "noteworthy",
                "is",
                "the",
                "use",
                "of",
                "the",
                "words",
                "\"",
                "aggressively",
                "seeking",
                "\"",
                "nuclear",
                ",",
                "biological",
                "or",
                "chemical",
                "weapons",
                "."
            ]

In [98]:
clean_text, offset_list = assemble_tokens(w_text)

In [99]:
clean_text[offset_list[40]:offset_list[42]]

'to describe '

In [388]:
clean_text[243:]

'.'

In [341]:
offset_list[20], offset_list[25]-1

(120, 137)

In [56]:
clean_text[offset_list[3]:offset_list[10]]

'as far as human rights are concerned'

In [74]:
for i in range(len(offset_list)-1):
    print(clean_text[offset_list[i]:offset_list[i+1]])

As 
for 
her 
wording 
regarding 
acquisition 
of 
weapons
, 
what 
is 
particularly 
noteworthy 
is 
the 
use 
of 
the 
words 
"
aggressively 
seeking
" 
nuclear
, 
biological 
or 
chemical 
weapons


In [97]:
w_text = [
                "PRESIDENT",
                "GEORGE",
                "Bush",
                "'s",
                "National",
                "Security",
                "Adviser",
                "Condoleezza",
                "Rice",
                "has",
                "recently",
                "\"",
                "defined",
                "\"",
                "the",
                "context",
                "and",
                "scope",
                "of",
                "the",
                "term",
                "\"",
                "axis",
                "of",
                "evil",
                "\"",
                "that",
                "Bush",
                "used",
                "in",
                "his",
                "State",
                "of",
                "the",
                "Union",
                "address",
                "a",
                "few",
                "weeks",
                "ago",
                "to",
                "describe",
                "Iraq",
                ",",
                "Iran",
                "and",
                "North",
                "Korea",
                "."
            ]

In [ ]:
def main():
    

In [ ]:


if __name__ == '__main__':
    main()